In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
delaney_chemberta2_valid2 = pd.read_csv('./chemberta2/results/delaney/chemberta2_valid2_delaney_3_predictions.csv')
delaney_molformer_valid2 = pd.read_csv('./molformer/results/delaney/molformer_valid2_delaney_3_99.csv')
delaney_molbert_valid2 = pd.read_csv('./molbert/results/delaney/molbert_valid2_delaney_3.csv')

# Load the test data for each model
delaney_chemberta2_test = pd.read_csv('./chemberta2/results/delaney/chemberta2_test_delaney_3_predictions.csv')
delaney_molformer_test = pd.read_csv('./molformer/results/delaney/molformer_test_delaney_3_99.csv')
delaney_molbert_test = pd.read_csv('./molbert/results/delaney/molbert_test_delaney_3.csv')

train_mean = -3.0955443786982246
train_sd = 2.121246879189704

# features

# Load the features from chemberta
delaney_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/delaney/chemberta2_valid2_delaney_3_features.csv')
delaney_chemberta2_features_test = pd.read_csv('./chemberta2/features/delaney/chemberta2_test_delaney_3_features.csv')

# Load the features from molformer
delaney_molformer_features_valid2 = pd.read_csv('./molformer/features/delaney/molformer_valid2_delaney_3_features.csv')
delaney_molformer_features_test = pd.read_csv('./molformer/features/delaney/molformer_test_delaney_3_features.csv')

# Load the features from molbert
delaney_molbert_features_valid2 = pd.read_csv('./molbert/features/delaney/molbert_valid2_delaney_3_features.csv')
delaney_molbert_features_test = pd.read_csv('./molbert/features/delaney/molbert_test_delaney_3_features.csv')

For delaney (regression)

In [2]:
# Preparing the actual and predicted values
# Chemberta2
delaney_chemberta_actual = delaney_chemberta2_test['target'] 
delaney_chemberta_pred = delaney_chemberta2_test['pred_raw']

# Molformer
delaney_molformer_actual = delaney_molformer_test['target']
delaney_molformer_pred = delaney_molformer_test['pred_raw']

# molbert
delaney_molbert_actual = delaney_molbert_test['target_raw']
delaney_molbert_pred = delaney_molbert_test['pred_raw']

In [3]:
# Calculating metrics
delaney_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", delaney_chemberta_actual, delaney_chemberta_pred),
                                 ("Molformer", delaney_molformer_actual, delaney_molformer_pred),
                                 ("Molbert", delaney_molbert_actual, delaney_molbert_pred)]:
    delaney_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

delaney_metrics_results

{'Chemberta2': {'MAE': 0.5606664320638193,
  'RMSE': 0.7305590849172536,
  'R2 Score': 0.8573821186737384,
  'Correlation': 0.9294289768725041},
 'Molformer': {'MAE': 0.4865245807831858,
  'RMSE': 0.662620137396756,
  'R2 Score': 0.8826744633917042,
  'Correlation': 0.9422375703567979},
 'Molbert': {'MAE': 0.5384853131061946,
  'RMSE': 0.6906433653118413,
  'R2 Score': 0.8725408620066292,
  'Correlation': 0.9387197751694945}}

In [4]:
# standardized valid2 labels
delaney_y_ensemble_valid2 = (delaney_chemberta2_valid2['target'] - train_mean)/train_sd

# Create the features for the ensemble from the prediction probabilities of being in class 1
delaney_X_ensemble_valid2 = pd.concat([
    delaney_chemberta2_valid2['pred_z'] - delaney_y_ensemble_valid2,
    delaney_molformer_valid2['pred_z'] - delaney_y_ensemble_valid2, 
    delaney_molbert_valid2['pred_z'] - delaney_y_ensemble_valid2,
    # add features from training set
    delaney_chemberta2_valid2['pred_z'],
    delaney_molformer_valid2['pred_z'],
    delaney_molbert_valid2['pred_z']
], axis=1)

# change feature names of the ensemble so that they are unique
delaney_X_ensemble_valid2.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Compute residuals
chemberta_y_residual = delaney_chemberta2_valid2['pred_z'] - delaney_y_ensemble_valid2
molformer_y_residual = delaney_molformer_valid2['pred_z'] - delaney_y_ensemble_valid2
molbert_y_residual = delaney_molbert_valid2['pred_z'] - delaney_y_ensemble_valid2

# For input features, we use pred_z and the other features
chemberta_X = pd.concat([delaney_chemberta2_valid2['pred_z'], 
                         delaney_chemberta2_features_valid2.iloc[:, 2:]], axis=1)
molformer_X = pd.concat([delaney_molformer_valid2['pred_z'], 
                         delaney_molformer_features_valid2.iloc[:, 1:]], axis=1)
molbert_X = pd.concat([delaney_molbert_valid2['pred_z'], 
                       delaney_molbert_features_valid2.iloc[:, 1:]], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Ensure that X_scaled and y_residual are numpy arrays
chemberta_X_scaled = np.array(chemberta_X_scaled)
molformer_X_scaled = np.array(molformer_X_scaled)
molbert_X_scaled = np.array(molbert_X_scaled)

chemberta_y_residual = np.array(chemberta_y_residual)
molformer_y_residual = np.array(molformer_y_residual)
molbert_y_residual = np.array(molbert_y_residual)

torch.manual_seed(0)

# Define custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output is continuous, no activation function
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params, X, y):
    kf = KFold(n_splits=5)
    rmses = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                      torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()  # Use RMSELoss for regression
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):  # Set number of epochs
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            rmse = np.sqrt(mean_squared_error(y_val_tensor.numpy(), outputs.numpy()))
            rmses.append(rmse)

    avg_rmse = np.mean(rmses)
    return {'loss': avg_rmse, 'status': STATUS_OK}  # Minimize RMSE

# Hyperparameter space for optimization
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Scaled input data (after standardization) and corresponding target (binary cross-entropy loss)
datasets = {
    'chemberta': (chemberta_X_scaled, chemberta_y_residual),
    'molformer': (molformer_X_scaled, molformer_y_residual),
    'molbert': (molbert_X_scaled, molbert_y_residual)
}

# Run Bayesian optimization for each model
best_params = {}
trials_dict = {}

for dataset_name, (X_scaled, y_residual) in datasets.items():
    print(f"Optimizing for {dataset_name}...")
    trials = Trials()
    
    best_params[dataset_name] = fmin(fn=lambda params: objective(params, X_scaled, y_residual),
                                     space=space,
                                     algo=tpe.suggest,
                                     max_evals=50,
                                     trials=trials,
                                     rstate=np.random.default_rng(0),  # Seed for reproducibility in hyperopt
                                     early_stop_fn=no_progress_loss(10))
    
    trials_dict[dataset_name] = trials
    print(f"Best hyperparameters for {dataset_name}: {best_params[dataset_name]}")

Optimizing for chemberta...
 24%|██▍       | 12/50 [00:31<01:40,  2.63s/trial, best loss: 0.3266269564628601]
Best hyperparameters for chemberta: {'dropout_rate': 0.2844084112121386, 'learning_rate': 0.0002374377408888569, 'num_layers': 2.0, 'num_neurons': 215.0}
Optimizing for molformer...
 32%|███▏      | 16/50 [00:49<01:44,  3.09s/trial, best loss: 0.3128936290740967]
Best hyperparameters for molformer: {'dropout_rate': 0.3476486742192898, 'learning_rate': 0.004207657605324512, 'num_layers': 3.0, 'num_neurons': 256.0}
Optimizing for molbert...
 62%|██████▏   | 31/50 [01:51<01:08,  3.61s/trial, best loss: 0.3046363294124603]
Best hyperparameters for molbert: {'dropout_rate': 0.47779937270829387, 'learning_rate': 0.0019521507369494975, 'num_layers': 4.0, 'num_neurons': 198.0}


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import mean_squared_error

# Custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output layer
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Function to train the model using the best hyperparameters
def train_model(X_train, y_train, best_params):
    # Create DataLoader for training data
    train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                  torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Initialize the model with the best hyperparameters
    model = SimpleNN(input_size=X_train.shape[1],
                     num_layers=int(best_params['num_layers']),
                     num_neurons=int(best_params['num_neurons']),
                     dropout_rate=best_params['dropout_rate'])
    
    # Define the loss and optimizer
    criterion = RMSELoss()
    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])

    # Train the model
    model.train()
    for epoch in range(100):  # Training for 100 epochs
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    return model

# Load your best hyperparameters from the optimization step
# Replace these with the actual best hyperparameters you found
best_params_chemberta = best_params['chemberta']
best_params_molformer = best_params['molformer']
best_params_molbert = best_params['molbert']

# Train the models for each dataset
best_model_chemberta = train_model(chemberta_X_scaled, chemberta_y_residual, best_params_chemberta)
best_model_molformer = train_model(molformer_X_scaled, molformer_y_residual, best_params_molformer)
best_model_molbert = train_model(molbert_X_scaled, molbert_y_residual, best_params_molbert)

In [7]:
# ChemBERTa
chemberta_X_test = pd.concat([delaney_chemberta2_test['pred_z'], 
                              delaney_chemberta2_features_test.iloc[:, 2:]], axis=1)

# MolFormer
molformer_X_test = pd.concat([delaney_molformer_test['pred_z'], 
                              delaney_molformer_features_test.iloc[:, 1:]], axis=1)

# MolBERT
molbert_X_test = pd.concat([delaney_molbert_test['pred_z'], 
                            delaney_molbert_features_test.iloc[:, 1:]], axis=1)

# Scale the test data using the corresponding scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Ensure that X_scaled and y_residual are numpy arrays
chemberta_X_test_scaled = np.array(chemberta_X_test_scaled)
molformer_X_test_scaled = np.array(molformer_X_test_scaled)
molbert_X_test_scaled = np.array(molbert_X_test_scaled)

# Predict on the test sets (assuming your test sets are already preprocessed and scaled)
with torch.no_grad():
    chemberta_X_test_tensor = torch.tensor(chemberta_X_test_scaled.astype(np.float32))
    molformer_X_test_tensor = torch.tensor(molformer_X_test_scaled.astype(np.float32))
    molbert_X_test_tensor = torch.tensor(molbert_X_test_scaled.astype(np.float32))
    
    chemberta_pred_test = best_model_chemberta(chemberta_X_test_tensor).numpy()
    molformer_pred_test = best_model_molformer(molformer_X_test_tensor).numpy()
    molbert_pred_test = best_model_molbert(molbert_X_test_tensor).numpy()
        
    # Ensure the predictions are 1-dimensional by flattening if necessary
    chemberta_pred_test = chemberta_pred_test.flatten()
    molformer_pred_test = molformer_pred_test.flatten()
    molbert_pred_test = molbert_pred_test.flatten()
    
# The predictions are now stored in chemberta_pred_test, molformer_pred_test, molbert_pred_test

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='residual_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='residual_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='residual_molbert')

# Load test data into ensemble DataFrame, adding the predicted residuals and original predictions
delaney_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # residual for Chemberta
    molformer_pred_test_series,                     # residual for Molformer
    molbert_pred_test_series,                       # residual for Molbert
    delaney_chemberta2_test['pred_z'],
    delaney_molformer_test['pred_z'],  
    delaney_molbert_test['pred_z']
], axis=1)

# Rename feature columns so that they are unique
delaney_X_ensemble_test.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

# True test labels
delaney_y_ensemble_test = delaney_chemberta2_test['target']

In [8]:
# scale the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
delaney_X_ensemble_valid2_scaled = scaler.fit_transform(delaney_X_ensemble_valid2)
delaney_X_ensemble_test_scaled = scaler.transform(delaney_X_ensemble_test)

delaney_X_ensemble_valid2_scaled = pd.DataFrame(delaney_X_ensemble_valid2_scaled, columns=delaney_X_ensemble_valid2.columns)
delaney_X_ensemble_test_scaled = pd.DataFrame(delaney_X_ensemble_test_scaled, columns=delaney_X_ensemble_test.columns)

In [9]:
delaney_X_ensemble_valid2_selected = delaney_X_ensemble_valid2_scaled
delaney_X_ensemble_test_selected = delaney_X_ensemble_test_scaled

# check shapes
print(delaney_X_ensemble_valid2_selected.shape)
print(delaney_X_ensemble_test_selected.shape)

(226, 6)
(113, 6)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Define RMSE loss
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model for regression
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Hyperparameter space with hp.quniform for integer distribution
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Global dataset variables assumed to be defined externally
X = delaney_X_ensemble_valid2_selected
y = delaney_y_ensemble_valid2

# Objective function for Bayesian optimization
def objective(params):
    params['num_layers'] = int(params['num_layers'])  # Ensure num_layers is an integer
    params['num_neurons'] = int(params['num_neurons'])  # Ensure num_neurons is an integer
    kf = KFold(n_splits=5)
    rmse_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Convert DataFrame to numpy arrays before making them PyTorch tensors
        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=params['num_layers'],
                         num_neurons=params['num_neurons'], dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = model(torch.tensor(X_val.values.astype(np.float32))).squeeze(1)
            val_targets = torch.tensor(y_val.values.astype(np.float32))
            rmse = np.sqrt(mean_squared_error(val_targets.numpy(), val_preds.numpy()))
            rmse_scores.append(rmse)

    avg_rmse = np.mean(rmse_scores)
    return {'loss': avg_rmse, 'status': STATUS_OK} # Minimize RMSE

# Run Bayesian optimization
trials = Trials()
delaney_nn_best_params = fmin(fn=objective,
                           space=space,
                           algo=tpe.suggest,
                           max_evals=50,
                           trials=trials,
                           rstate=np.random.default_rng(0),  # Seed for hyperopt
                           early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", delaney_nn_best_params)


 64%|██████▍   | 32/50 [01:12<00:40,  2.27s/trial, best loss: 0.019722644239664078]
Best hyperparameters: {'dropout_rate': 0.02494977934475137, 'learning_rate': 0.00150761351727997, 'num_layers': 1.0, 'num_neurons': 245.0}


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Define a function to compute RMSE
def compute_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert parameters to the correct format if necessary
delaney_nn_best_params = {
    'num_layers':  int(delaney_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons':  int(delaney_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': delaney_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': delaney_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(delaney_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(delaney_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(delaney_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(delaney_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=delaney_X_ensemble_valid2_selected.shape[1], num_layers=delaney_nn_best_params['num_layers'],
                         num_neurons=delaney_nn_best_params['num_neurons'], dropout_rate=delaney_nn_best_params['dropout_rate'])
criterion = RMSELoss()
optimizer = optim.Adam(model.parameters(), lr=delaney_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = outputs.squeeze(1).numpy() * train_sd + train_mean

    # Calculate metrics
    mae = mean_absolute_error(y_test_tensor.numpy(), predictions)
    rmse = compute_rmse(y_test_tensor.numpy(), predictions)
    r2 = r2_score(y_test_tensor.numpy(), predictions)
    correlation, _ = pearsonr(y_test_tensor.numpy().squeeze(1), predictions)

    delaney_nn_metrics = {
        'MAE': mae,
        'RMSE': rmse,
        'R2 Score': r2,
        'Correlation': correlation
    }

delaney_nn_metrics

{'MAE': 0.46610302,
 'RMSE': 0.63075316,
 'R2 Score': 0.8936880230903625,
 'Correlation': 0.9456342535157194}

In [12]:
# create a table to record all metrics for delaney
delaney_metrics_results["Neural Network"] = delaney_nn_metrics

delaney_metrics_df = pd.DataFrame(delaney_metrics_results).T
# keep 3 digits after the decimal point
delaney_metrics_df = delaney_metrics_df.round(3)

# export table to csv
delaney_metrics_df.to_csv('./split3_delaney_metrics_nn.csv')